In [1]:
from bs4 import BeautifulSoup 
import re
from html import unescape
import requests
import json
from functools import reduce
import time

In [2]:
def download(method,url,param=None,data=None, timeout=5, maxretries=10):
    headers={
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36",
        "x-client-data": "CI62yQEIpbbJAQjEtskBCKmdygEIqKPKAQjiqMoBCJetygEIza3KAQjKr8oB"}
                #  개발자 도구에서 qql의 request헤더부분 설정함
    try:
        resp=requests.request(method, url,params=param,data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<=e.response.status_code<600 and maxretries>0:
            print(maxretries)
            
            download(method,url,param,data,timeout,maxretries-1)
        elif 400<=e.response.status_code<500 :
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [20]:
def youtube_jamak(url_id,lst=None):
    if lst==None:
        lst=[]
    headers={
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36",
        "x-client-data": "CI62yQEIpbbJAQjEtskBCKmdygEIqKPKAQjiqMoBCJetygEIza3KAQjKr8oB"}
    id=url_id
    base = 'https://www.youtube.com/watch?v=url_id'
    jamak_url_base='https://www.youtube.com/api/timedtext?v=url_id&asr_langs=de%2Cen%2Ces%2Cfr%2Cit%2Cja%2Cko%2Cnl%2Cpt%2Cru&caps=asr&xorp=true&hl=ko&ip=0.0.0.0&ipbits=0&expire=expire&sparams=ip%2Cipbits%2Cexpire%2Cv%2Casr_langs%2Ccaps%2Cxorp&signature=signature&key=yt8&kind=asr&lang=ko&fmt=srv3'
    video_url = base.replace(requests.compat.urlparse(base)[4],'v={}'.format(url_id))
    jamak_url=jamak_url_base.replace(requests.compat.urlparse(jamak_url_base)[4].split('&')[0],'v={}'.format(url_id))
    html = download('get',video_url)
    dom = BeautifulSoup(html.text, 'lxml')
    res = [re.findall(r'signature=(.+?)(\\\\u0026|\\u0026)', _.text) for _ in dom.select('script') if "signature" in _.text]
    result = [x[0] for _ in res for x in _]
    change_url = jamak_url.replace(requests.compat.urlparse(jamak_url)[4].split('&')[9],'signature={}'.format(result[0]))
    change_url=change_url.replace(requests.compat.urlparse(jamak_url)[4].split('&')[7],re.findall(r'expire=\d+',html.text)[-1])
    try:
        resp=requests.request('get', change_url,headers=headers)
        dom =BeautifulSoup(resp.text,'lxml')
        lst.append(dom)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 400<=e.response.status_code<500 :
            youtube_jamak(id,lst)
    return [_.text for _ in lst[len(lst)-1].find_all('s')]

In [23]:
res=youtube_jamak('YIHG7U_lC7A')

In [24]:
ls = []
text = [x for x in [res[_:_+8] if _ %8 ==0 else None for _ in range(len(res))] if x != None]
jamak = [reduce(lambda x,y: x+y,t) for t in text]
[ls.append([x])for x in jamak]
ls

[['신상 아시시 그게 일부 매우 를 작은 에'],
 [' 전화 간답니다탈모 따서 있다라고이 시국에 7'],
 [' 매너를 방송에 출연시켜 다뇨 닮았다 아닐까이제 뭐'],
 [' 요즘에야 악마 뭐인사를 먼저 뜨니까 그래도 미상'],
 [' 보세요안녕하세요 오늘 하루 담가 봤습니까 저는이은'],
 [' 단백 입니다 안녕하세요 구유 마족 우유 마초의 아이'],
 [' 대처 완전지우기 들어왔어요 4 지우기 로 가지고'],
 [' 갔죠 아닌데 그 얼마나 쌀 기록말살을 닥치는'],
 [' 기록 - 정책을 하고 있는데요 입맞춤에아예 저는'],
 [' 피우게 재앙이 싫어 중어 요즘에 그걸로도 활동하시고 예'],
 [' 그 뭐옷 갈아입히기 그 아 스타일리스트 l'],
 [' 리스트 스타일리스트로 하고 있습니다연예인스타일 비스트 하시고 예'],
 [' 4 매드클라운 음 스타일리스트 하고 있어요붙잡지 백신'],
 [' 4거기 를 퇴사한 제 거래한 3 그냥'],
 [' 님의 그다음 회사도 있잖아요 4플레이보이 라고 보기'],
 [' 면제 지면은 내리고 맥심 플레이 vt 에서의 에'],
 ['전에 블링 v 블링 테다 타서 했죠'],
 [' 여기서 이렇게 퇴사 할 만하지그러니까 원래 좀'],
 [' 에디터 라는 직업이 이직이 빠르더라구요 순환 유리빨라요'],
 [' 음 예이게 좀 길지가 않고 만에 후임이'],
 [' 담아 트임마트 이미 얻는데그 오 예'],
 [' 아 예분반 도의 저러면 진짜봐바 허가받아'],
 [' 버린다 구라고 여기 이해가너 지금 내 앞에서도'],
 [' 이러니 식량 등의 무엇이 메 힘이 1심에 합'],
 [' 뜯어몰라 그냥 아니 물 드세요이거 알아서'],
 [' 옆에서 제지 하세요 예 지나면 바로 먹었어요 아닌데'],
 [' 어 오기전에 이거 하지 말라고 얘기 했잖아요'],
 ['바로 통해서도 개지랄 했던것 같아요 이거 이거 보라고'],
 [' 이거 에 말이 거병어 가야된다고가끔 제가'],
 [' 이렇게 사진도 찍어 줘요 그 행위의 하도 마우스를'],
 [' 해